# Monte Carlo under Expected Utlity and Risk Neutrality

In [1]:
from prepare import *
ecs=equalize_and_compress_and_sort


current path C:\Users\aes05kgb\Kelvin\Kel1\Risk\Stan Example\Stan_basic


In [2]:
#Load the compied code
sm=loadmodel('modelbasic')

In [3]:
#The data contains 14300 rows and 45 columns, it has different treatments etc but we will not use these
dat=pd.read_excel('Fulldata3.xlsx')

In [4]:
#Create a variable specifying the domain of the gamble as 1,2,3 below
dat['Dom']=(dat['Gains']==1)*1+(dat['Loss']==1)*2+(dat['Mixed']==1)*3

#All lotteries have up to three payoffs. However, there are missing values that should be given zeros
xA=twodma(dat[['prize1l','prize2l','prize3l']].fillna(0))
pA=twodma(dat[['p1left','p2left','p3left']].fillna(0))
pA=pA/sumc(pA.T)
xB=twodma(dat[['prize1r','prize2r','prize3r']].fillna(0))
pB=twodma(dat[['p1right','p2right','p3right']].fillna(0))
pB=pB/sumc(pB.T)

#The lotteries are not ordered and can contain repeated values, 
#The ecs function (written in python) compressess them into unique value and orders 
xA_,pA_=ecs(xA,pA)
xB_,pB_=ecs(xB,pB)

#Create the cumulative and decumulative vectors for options A and B
PA=cumdist(pA_)
FA=dcumdist(pA_)
PB=cumdist(pB_)
FB=dcumdist(pB_)

Y=array(-dat['choices']+2) #relabel the choices 1 for A and 2 for B, originally 0 and 1
DOM=array(dat['Dom'])
ideas=sorted(list(set(dat['id'])))

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Below we generate preferences for these lotteries based on expected value. Therefore, all the parameters have means of one

In [5]:
ea=pA_*xA_
eb=pB_*xB_

ea=sumc(ea.T)
eb=sumc(eb.T)
[shape(ea),shape(eb)]

prob=exp(ea-eb)/(1+exp(ea-eb))
s=(prob>twodma(np.random.random(size=rows(prob)))).astype(int)
Y=array(-s+2)
#sumc(s)/rows(Y)
meanc(prob)

array([[0.69909552]])

In [6]:
#Now create the arrays that can be read in a format that can be put into STAN
#The dimensions will match that in the Stan Code
y,xa,xb,fa,fb,pa,pb,dom=[],[],[],[],[],[],[],[]


M=100  #The number of tasks performed by each person

#Data is organised into responses by individuals
j=0
for i in ideas:
    y+=[Y[j:j+M]]      #The Choices
    xa+=[xA_[j:j+M,:]] #Lottery A Payoffs
    xb+=[xB_[j:j+M,:]] #Lottery B Payoffs
    fa+=[FA[j:j+M,:]]  #DeCumulative distribution for A
    fb+=[FB[j:j+M,:]]  #DeCumulative distribution for B
    pa+=[PA[j:j+M,:]]  #Cumulative distribution for A 
    pb+=[PB[j:j+M,:]]  #Cumulative distribution for B
    dom+=[DOM[j:j+M]]  #Specify the domain of the lottery
    j=j+M
    
N=3           #The potential number of payoffs
T=len(ideas)  #The number of people
y=squeeze(y)

In [7]:
#You can see the dimensions here
[shape(y),shape(xa),shape(xb),shape(fa),shape(fb),shape(pa),shape(pb),shape(dom)]

[(143, 100),
 (143, 100, 3),
 (143, 100, 3),
 (143, 100, 3),
 (143, 100, 3),
 (143, 100, 3),
 (143, 100, 3),
 (143, 100)]

In [7]:
sd=0.33  #The standard deviation for the prior mean of the key parameters

data={'typ': 1,  #Negative number if CE model otherwise utility
      'T':T,     #Number of people
      'N':N,      #Number of payoffs in each lottery
      'M':M,      #Number of choices made by an individual
      'y':y,         #Choices, 1 or 2, 
      'Domain':dom,  #Gains 1, Loss 2, Mixed 3.
      
      'xA':xa, # T by M array of N vectors 
      'xB':xb, # T by M array of N vectors 
      
      'FA':fa, # T by M array of N vectors    #Decumulative distribution of Lottery A 
      'PA':pa, # T by M array of N vectors    #Cumulative distribution of lottery A
    
      'FB':fb, # T by M array of N vectors    #Decumulative distribution of Lottery A
      'PB':pb, # T by M array of N vectors    #Decumulative distribution of Lotter B
      
      'm_alpha':1,'m_beta':1,'m_lamda':1,     #Prior neans of utility parameters
      'm_gama1':1,'m_gama2':1,                #Prior Means of prob weighting parameters in gain domain  
      'm_delta1':1,'m_delta2':1,              #Prior means of prob weighting parameters in loss domain
      'm_scale':0,                            #Prior mean log parameter governing vartiaion of error scale across domains
      
      's_alpha':sd,'s_beta':sd,'s_lamda':sd,  #prior sd of mean utility parameters (sd=standard deviation)
      's_gama1':sd,'s_gama2':sd,              #prior sd of mean prob weighting parameters in gain domain  
      's_delta1':sd,'s_delta2':sd,            #prior sd of mean prob weighting parameters in loss domain
      's_scale':3,                            #prior sd parameter for variation of error of error log scale across domains
      'ga':1, 'gb':0.1                        #Gamma parameters for the prior precision, for individual effects
                                              #Note that 1, 0.1 
                                              #roughly corresponds to mean standard deviation of 1/2,median=0.4 but lower mode 
                                               
     }

In [9]:
pars=('alpha','beta','lamda','scale','delta1','delta2','gama1','gama2','alpha_','beta_','lamda_','delta1_','delta2_',
      'gama1_','gama2_','scale_')
#fit = sm.sampling(data=data,iter=1500,warmup=500, chains=8,thin=1,pars=pars)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [10]:
#You can only run this if you have just run the model. Note that it can take a very long time for high dimensional models
#This is because the Rhats etc are being computed
print(fit)

Inference for Stan model: anon_model_86e589b1fad10845e9c8d21a72660ac1.
8 chains, each with iter=1500; warmup=500; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=8000.

                 mean se_mean     sd   2.5%     25%     50%     75%  97.5%  n_eff   Rhat
alpha            1.01  1.3e-3   0.02   0.97     1.0    1.01    1.02   1.05    214   1.06
beta             0.98  9.0e-4   0.02   0.95    0.97    0.98     1.0   1.02    343   1.04
lamda            0.98  5.9e-3   0.07   0.84    0.93    0.98    1.03   1.12    152   1.08
scale[1]         0.03  5.5e-3   0.08  -0.14   -0.02    0.03    0.09   0.19    228   1.06
scale[2]        -0.07  3.7e-3   0.07  -0.21   -0.12   -0.07   -0.02   0.06    349   1.03
scale[3]       1.5e-4  3.8e-3   0.06  -0.13   -0.04  1.6e-3    0.04   0.12    278   1.05
delta1           1.02  2.3e-4   0.01   0.99    1.01    1.02    1.03   1.04   2683   1.01
delta2           0.99  5.9e-4   0.01   0.96    0.98    0.99     1.0   1.02    579   1.02
gama1      

In [9]:
#z=pull(fit)

In [12]:
#save the MCMC otuput if run by stan, you might want to hash this out once you have saved the model once
#this will go to the working directory
#savemodel(z,'mc_output')

In [10]:
#load the MCMCoutput run previously by stan
#The output must be in the working directory
z=loadmodel('mc_output')

In [11]:
#get the key mcmc output for the mean parameters
alpha=z['alpha']
beta=z['beta']
lamda=z['lamda']
scale=exp(z['scale'])
delta1=z['delta1']
delta2=z['delta2']
gama1=z['gama1']
gama2=z['gama2']


alpha=z['alpha']
beta=z['beta']
lamda=z['lamda']
scale=exp(z['scale'])
delta1=z['delta1']
delta2=z['delta2']
gama1=z['gama1']
gama2=z['gama2']


means_=frame(cc([twodma(alpha),twodma(beta),twodma(lamda),twodma(scale),twodma(delta1),twodma(delta2),twodma(gama1),twodma(gama2)]))
means_.columns=['alpha','beta','lamda','scale1','scale2','scale3','delta1','delta2','gama1','gama2']


# The output for the mean parameters is as below

In [12]:
means_.describe()

,alpha,beta,lamda,scale1,scale2,scale3,delta1,delta2,gama1,gama2
count,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,1.008379,0.984523,0.982849,1.034689,0.934560,1.002196,1.018578,0.990198,1.024672,1.001294
std,0.019332,0.016669,0.073019,0.085468,0.064960,0.063943,0.012161,0.014264,0.014590,0.017413
min,0.944826,0.927415,0.732650,0.758267,0.695447,0.770823,0.970815,0.932136,0.974161,0.922562
25%,0.995825,0.973059,0.933777,0.976722,0.889744,0.958914,1.010474,0.980559,1.014800,0.989780
50%,1.007883,0.983934,0.983138,1.033077,0.933414,1.001553,1.018473,0.989961,1.024565,1.001089
75%,1.020616,0.995308,1.029701,1.090761,0.977281,1.045402,1.026611,0.999657,1.034366,1.013130
max,1.081398,1.046126,1.293140,1.451831,1.192484,1.268224,1.062255,1.043282,1.079458,1.061892
